In [26]:
!pip install tensorflow-model-optimization

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [27]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

In [28]:
base_path = "/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition"
base_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition'

In [29]:
uci_har_dataset_path = f'{base_path}/data/UCI_HAR'
uci_har_dataset_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/data/UCI_HAR'

In [30]:
if base_path not in sys.path:
    sys.path.append(base_path) 

In [31]:
import src.load_uci_har_dataset as uci_har

In [32]:
Xtrain, ytrain, xtest, ytest, labels = uci_har.load_uci_har_dir(uci_har_dataset_path)

In [33]:
import src.get_models as models

In [34]:
import importlib
importlib.reload(models)

<module 'src.get_models' from '/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/src/get_models.py'>

In [35]:
base_model = keras.models.load_model(f'{base_path}/models/gru_model.keras')

2025-12-15 18:32:43.477060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:32:43.477963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:32:43.478563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [36]:
baseline_model_loss, baseline_model_accuracy = base_model.evaluate(
    xtest, ytest)

print('Baseline test accuracy:', baseline_model_accuracy)


2025-12-15 18:32:44.930365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:32:44.931844: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:32:44.932521: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

93/93 [==============================] - 2s 15ms/step - loss: 0.3158 - accuracy: 0.9165
Baseline test accuracy: 0.9165253043174744


In [37]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 64
epochs = 100
validation_split = 0.1

num_images = Xtrain.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(base_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 prune_low_magnitude_gru_9 (  (None, 128, 64)          27267     
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_gru_10   (None, 32)               18627     
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_dense_6  (None, 128)              8322      
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_batch_n  (None, 128)              513       
 ormalization_3 (PruneLowMag                               

2025-12-15 18:32:49.564906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:32:49.565829: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:32:49.566381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [38]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=f'{base_path}/pruned_models_gru'),
]

model_for_pruning.fit(Xtrain, ytrain,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/100


2025-12-15 18:32:58.651922: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:32:58.652911: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:32:58.653543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

104/104 [==============================] - ETA: 0s - loss: 0.1815 - accuracy: 0.9261

2025-12-15 18:33:08.074246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:33:08.075133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:33:08.075701: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

104/104 [==============================] - 10s 67ms/step - loss: 0.1815 - accuracy: 0.9261 - val_loss: 0.4304 - val_accuracy: 0.7894
Epoch 2/100
104/104 [==============================] - 6s 62ms/step - loss: 0.2619 - accuracy: 0.8953 - val_loss: 0.2752 - val_accuracy: 0.8859
Epoch 3/100
104/104 [==============================] - 6s 62ms/step - loss: 0.1907 - accuracy: 0.9231 - val_loss: 0.2889 - val_accuracy: 0.8777
Epoch 4/100
104/104 [==============================] - 6s 61ms/step - loss: 0.1842 - accuracy: 0.9256 - val_loss: 0.2124 - val_accuracy: 0.9266
Epoch 5/100
104/104 [==============================] - 6s 61ms/step - loss: 0.1697 - accuracy: 0.9332 - val_loss: 0.2277 - val_accuracy: 0.9212
Epoch 6/100
104/104 [==============================] - 7s 64ms/step - loss: 0.1585 - accuracy: 0.9349 - val_loss: 0.2266 - val_accuracy: 0.9090
Epoch 7/100
104/104 [==============================] - 6s 61ms/step - loss: 0.1663 - accuracy: 0.9339 - val_loss: 0.1979 - val_accuracy: 0.9239
Epo

104/104 [==============================] - 6s 57ms/step - loss: 0.0895 - accuracy: 0.9687 - val_loss: 0.1469 - val_accuracy: 0.9429
Epoch 58/100
104/104 [==============================] - 6s 62ms/step - loss: 0.0901 - accuracy: 0.9675 - val_loss: 0.1700 - val_accuracy: 0.9389
Epoch 59/100
104/104 [==============================] - 7s 66ms/step - loss: 0.0827 - accuracy: 0.9707 - val_loss: 0.2496 - val_accuracy: 0.9158
Epoch 60/100
104/104 [==============================] - 6s 60ms/step - loss: 0.0812 - accuracy: 0.9719 - val_loss: 0.2061 - val_accuracy: 0.9212
Epoch 61/100
104/104 [==============================] - 6s 62ms/step - loss: 0.0812 - accuracy: 0.9704 - val_loss: 0.1411 - val_accuracy: 0.9484
Epoch 62/100
104/104 [==============================] - 6s 62ms/step - loss: 0.0841 - accuracy: 0.9696 - val_loss: 0.1949 - val_accuracy: 0.9280
Epoch 63/100
104/104 [==============================] - 7s 63ms/step - loss: 0.0722 - accuracy: 0.9745 - val_loss: 0.2259 - val_accuracy: 0.923

In [39]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   xtest, ytest, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9165253043174744
Pruned test accuracy: 0.9043094515800476


In [40]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
pruned_keras_file = f'{base_path}/pruned_models_gru'
keras.models.save_model(model_for_export, pruned_keras_file , include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

2025-12-15 18:43:31.097202: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:31.098135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:31.098936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:31.779781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:31.780635: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:31.781206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:32.324902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128,6]
	 [[{{node inputs}}]]
2025-12-15 18:43:32.333638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128,6]
	 [[{{node inputs}}]]
2025-12-15 18:43:32.361181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_5' with dtype float and shape [?,128,6]
	 [[{{node input_5}}]]
2025-12-

2025-12-15 18:43:32.995148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:32.996130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:32.996812: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:33.656177: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_input_5' with dtype float and shape [?,128,6]
	 [[{{node serving_default_input_5}}]]


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_gru/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_gru/assets


Saved pruned Keras model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_gru


In [41]:
import tensorflow as tf
run_model = tf.function(lambda x: model_for_pruning(x))
BATCH_SIZE = 1
STEPS = 128
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model_for_pruning.inputs[0].dtype))

# model directory.
MODEL_DIR = f"{base_path}/GRU_Pruning"
model_for_pruning.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

2025-12-15 18:43:34.126090: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:34.127198: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:34.127815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:35.478543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:35.479706: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:35.480652: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:36.494282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:36.495316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:36.495988: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:37.486437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:43:37.487267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:43:37.487853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning/assets


In [42]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
pruned_tflite_model = converter.convert()

pruned_tflite_file = f"{base_path}/models/gru_pruned.tflite"

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

2025-12-15 18:43:39.040938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:43:39.041025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:43:39.041063: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:39.793185: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:43:39.793265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:43:39.793302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:40.225088: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:43:40.225162: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:43:40.225194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Saved pruned TFLite model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/models/gru_pruned.tflite


2025-12-15 18:43:40.990770: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 18:43:40.990804: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 18:43:40.991447: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning
2025-12-15 18:43:41.003624: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 18:43:41.003659: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning
2025-12-15 18:43:41.048382: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-15 18:43:41.168586: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path:

In [43]:
import tempfile
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [44]:
keras_file = f"{base_path}/models/gru_baseline.h5"
keras.models.save_model(base_model, keras_file, include_optimizer=False)

In [45]:
print("Size of gzipped TFlite model: %.2f bytes" % (get_gzipped_model_size(keras_file)))

Size of gzipped TFlite model: 40544.00 bytes


In [46]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = f"{base_path}/models/gru_pruned_dynamic.tflite"

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

2025-12-15 18:43:41.785129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:43:41.785271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:43:41.785353: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:42.253105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:43:42.253181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:43:42.253215: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:43:43.066020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:43:43.066099: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:43:43.066134: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Saved quantized and pruned TFLite model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/models/gru_pruned_dynamic.tflite
Size of gzipped pruned and quantized TFlite model: 51857.00 bytes
Size of gzipped pruned TFlite model: 51831.00 bytes


2025-12-15 18:43:43.774800: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 18:43:43.774860: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 18:43:43.775551: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning
2025-12-15 18:43:43.788532: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 18:43:43.788578: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning
2025-12-15 18:43:43.831911: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-15 18:43:43.950775: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path:

In [47]:
def evaluate_model(interpreter):
    in_det  = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]
    in_idx  = in_det["index"]
    out_idx = out_det["index"]
    in_dtype = in_det["dtype"]     
    in_scale, in_zero = in_det["quantization"]
    print("dtype", in_dtype)

    prediction_digits = []
    for i, test_image in enumerate(xtest):
        if i % 1000 == 0:
            print(f"Evaluated on {i} results so far.")

        xi = np.expand_dims(test_image, axis=0).astype(np.float32)

        if in_dtype == np.int8:
            xi_q = np.clip(np.round(xi / in_scale + in_zero), -128, 127).astype(np.int8)
            interpreter.set_tensor(in_idx, xi_q)
        elif in_dtype == np.uint8:
            xi_q = np.round(xi / in_scale + in_zero).astype(np.uint8)
            interpreter.set_tensor(in_idx, xi_q)
        else:
            interpreter.set_tensor(in_idx, xi)

        interpreter.invoke()

        yo = interpreter.get_tensor(out_det["index"])[0]
        prediction_digits.append(int(np.argmax(yo)))

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == ytest).mean()
    return accuracy

In [48]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

dtype <class 'numpy.float32'>
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9043094672548354
Pruned TF test accuracy: 0.9043094515800476


In [49]:
print("GRU Pruned model size in MB:", os.path.getsize(f"{base_path}/models/gru_pruned.tflite") / float(2**20))

GRU Pruned model size in MB: 0.2696990966796875


In [50]:
print("GRU Pruned and quantized model size in MB:", os.path.getsize(f"{base_path}/models/gru_pruned_dynamic.tflite") / float(2**20))

GRU Pruned and quantized model size in MB: 0.2697029113769531
